## **Pasos Clustering**

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt 
import matplotlib.cm as cm 
import numpy as np
from scipy import stats

In [ ]:
import pandas as pd
df = pd.read_csv(path_al_archivo)

INSPECCIONAR: ver que columnas tiene, que tipos de datos, cuantos nulos.

In [ ]:
df.info()
# Averigua la distribucion general de la tabla y nos dice los nombres de las columnas 

In [ ]:
df.isnull().sum() 
#Muestra cuantos datos nulos hay por columna

In [ ]:
sns.heatmap(datos.isnull(), cmap = "viridis") 
#Grafico de datos nulos
#Hace la inspección de los datos faltantes de forma visual.

CARACTERIZAR: DESCRIBE, saber cual es la media, mínima

In [ ]:
df.describe()
#Análisis estadístico básico del conjunto de datos

Podemos calcular la frecuencia de aparición de cada categoría/datos.

In [ ]:
df['columna'].value_counts()
#Cuenta cuántas veces del total de filas apparece cada una de ellas

In [ ]:
sns.pairplot(df)

**Selección de columnas numéricas**

In [ ]:
columnas = list(datos.columns)

In [ ]:
del(columnas[:2]
columnas
#Miramos el codigo anterior, buscamos las columnas no numericas y las eliminamos de COLUMNAS

In [ ]:
def verificacion_de_medias(lista):
    test = {}
    for columna in lista:
        w , p = stats.shapiro(datos[columna].dropna())
        test[columna] = p
    return test
# Realiza el test de normal para cada una de las columnas

In [ ]:
verificacion_de_medias(columnas) 
#TIENE QUE DAR 0.05 O MAS PARA QUE ESTE NORMALIZADA

## **Limpiar el dataframe de las 3 posibles anomalias**


PASOS EN EL CASO QUE SEA NECESARIO REEMPLAZAR LOS NAN

1) Saco los duplicados

2) Outliers

3) Recién ahí puedo calcular la media “limpia” si necesitara rellenar los NaN

**OUTLIERS**

In [ ]:
# COMO CALCULAR LOS OUTLIERS!!

Cs1 = datos["Actividad (en hs)"].quantile(0.95)
Ci1 = datos["Actividad (en hs)"].quantile(0.05)
Cs2 = datos["Acceso a Instagram (en hs)"].quantile(0.95)
Ci2 = datos["Acceso a Instagram (en hs)"].quantile(0.05)
#Averiguo el valor más chico y más grande de cada columna (los cuantiles)

datos1 = datos[(datos["Actividad (en hs)"] >= Ci1) & (datos["Actividad (en hs)"] <= Cs1) & (datos["Acceso a Instagram (en hs)"] >= Ci2) & (datos["Acceso a Instagram (en hs)"] <= Cs2)]
datos1
# En la última fórmula quedate con los datos que se encuentren entre los cuantiles que calculamos.

CREAMOS UN NUEVO DATAFRAME (variable datos1) SIN LOS OUTLIERS

**DATOS NULOS**

- Eliminar los valores nulos

In [ ]:
datos2 = datos1.dropna().reset_index(drop=True) 
datos2

In [ ]:
datos1.dropna(inplace=True).reset_index(drop=True) 
#si quiero eliminar los datos nulos directamente sobre datos 1 uso este metodo

- Rellenar los valores nulos con la media

In [ ]:
df.fillna(df["columna_con_faltantes"].mean(), inplace= True)
# Columna_con_faltantes --> columna con muchos datos nulos

**ELIMINAR DATOS DUPLICADOS**

In [ ]:
datos3 = datos2.drop_duplicates().reset_index(drop=True)
datos3

In [ ]:
datos1.drop_duplicates(inplace=True).reset_index(drop=True)

In [ ]:
sns.pairplot(datos3)
#luego de toda la limpieza deberia empezar a ver una correlacion entre grupos

## **NORMALIZACIÓN**

In [ ]:
scaler = StandardScaler()
datos_escalado = scaler.fit_transform(datos3[columnas])
#Es importa ver que selecciona COLUMNAS --> es decir las columnas numericas que seleccione antes!!!

## **CLUSTERING**

**Cálculo de inercias**

In [ ]:
def inercias_por_k(df_escalado):
  inercias = {}
  for i in range(1,11):
      kmeans = KMeans(n_clusters = i, init="random", n_init=10, max_iter=300, random_state=123457)
      kmeans.fit(df_escalado)
      inercias[i] = kmeans.inertia_
  return inercias

#Me da por cada grupo el valor de inercia

In [ ]:
inercias = inercias_por_k(datos_escalado)
inercias

In [ ]:
df_inercias = pd.DataFrame(inercias.items(), columns=["K", "inercia"])
df_inercias
#Convierto en un dataframe las inercias

**GRAFICO DEL CODO**

In [ ]:
sns.pointplot(data = df_inercias, x = "K", y = "inercia")

El silhouette es quien verificaba que la selección de grupos sea la correcta.

In [ ]:
k=2 
# El 2 es la opcion optima de k elegida gracias al grafico del codo
kmeans = KMeans(n_clusters = k, init="random", n_init=10, max_iter=300, random_state=123457)
kmeans.fit(datos_escalado)

In [ ]:
print(kmeans.inertia_)
# Da la incercia total

**Grafico del clustering**

In [ ]:
colores = ["red", "green"]
g = sns.scatterplot(x = datos_escalado[:,2], y = datos_escalado[:, 0], palette = colores, hue = kmeans.labels_, alpha = 0.5)
g = sns.scatterplot(x = kmeans.cluster_centers_[:,2], y = kmeans.cluster_centers_[:,0], zorder = 10, palette = colores, hue=[0,1], legend = False, marker=6, s=200)
#Para ver que pongo en los corchetes, voy a ver lo que me devuelve COLUMNAS y elijo la posicion en base a lo que yo quiero evaluar.

In [ ]:
silhouette_avg = silhouette_score(datos_escaleado, kmeans.labels_)
sample_silhouette_values = silhouette_samples(datos_escaleado, kmeans.labels_)

def graficarSilhouette (k, labels, sample_silhouette_values, silhouette_avg):
  fig, ax1 = plt.subplots(1, 1)
  y_lower = 10
  for i in range(k):
      ith_cluster_silhouette_values = \
          sample_silhouette_values[labels == i]

      ith_cluster_silhouette_values.sort()

      size_cluster_i = ith_cluster_silhouette_values.shape[0]
      y_upper = y_lower + size_cluster_i

      color = cm.nipy_spectral(float(i) / k)
      ax1.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values, facecolor=color, edgecolor=color, alpha=0.7)
      ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
      y_lower = y_upper + 10

  ax1.set_title("Plot del silhouette de cada cluster")
  ax1.set_xlabel("Coeficiente de silhouette")
  ax1.set_ylabel("Etiqueta del cluster")
  ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
  ax1.set_yticks([])

graficarSilhouette (k, kmeans.labels_, sample_silhouette_values, silhouette_avg)

Me dice si la división en grupos que realicé es correcta o no. Mientras más cercano a 1 mejor agrupamiento.


Valores negativos: hay datos agrupados en el grupo 0 que no deberían estar ahí.